In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold
from matplotlib import pyplot
from statistics import median
import pickle
import csv
import warnings
import datetime
import multiprocess
import os
warnings.filterwarnings("ignore")

In [2]:
project_list = ['heroku.csv', 'rails.csv', 'gradle.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'vagrant.csv', 'rubinius.csv', 'open-build-service.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'puppet.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv', 'graylog2-server.csv']


In [3]:
fi_dict = {}

In [4]:
for p in project_list:
    p_name = p.split('.')[0]
    fi_dict[p] = {}
    path = 'dump_data/' + p_name + '_models/'
    contents = os.listdir(path)
    
    for f in contents:
        model_file = path + f
        with open(model_file, 'rb') as file:
            model = pickle.load(file)
        
        #print(p, ver)
        
        
        important_features_dict = {}
        for idx, val in enumerate(model.feature_importances_):
            important_features_dict[idx] = val
        
        important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)
        
        #print(important_features_list)
        
        fi_dict[p][ver] = {}
        fi_dict[p][ver]['list'] = important_features_list
        fi_dict[p][ver]['value'] = important_features_dict

FileNotFoundError: [Errno 2] No such file or directory: 'dump_data/heroku_models/'

In [77]:
feature_names = ['git_diff_src_churn', 'gh_num_commit_comments', 'git_diff_test_churn', 'gh_diff_files_added', 'gh_diff_files_deleted', 'gh_diff_files_modified', 'git_num_all_built_commits', 'gh_num_commits_on_files_touched', 'time_out']
combined_for_project = {}
values_for_projects = {}


In [78]:
for p in project_list:
    data = fi_dict[p]
    flist = [[], [], [], [], [], [], [], [], []]
    vals = [[], [], [], [], [], [], [], [], []]
    fi = 0
    
    for ver in range(1,11):
        for x in range(0,9):
            flist[x].append(data[ver]['list'][x])
            
    for ver in range(1, 11):  
        for x in range(0,9):
            vals[x].append(data[ver]['value'][x])
    
    final = [0,0,0,0,0,0,0,0,0]
    final_value = [0,0,0,0,0,0,0,0,0]
    
    for i in range(0, 9):
        final[i] = max(set(flist[i]), key = flist[i].count)
    
    for i in final:
        final_value[i] = median(vals[i])
    
    values_for_projects[p] = final_value
    combined_for_project[p] = final

In [79]:
#after manual interference
combined_for_project['vagrant.csv'][-1] = 1
combined_for_project['vagrant.csv'][-2] = 4
combined_for_project['graylog2-server.csv'][2] = 0
combined_for_project['graylog2-server.csv'][-1] = 1

In [80]:
final_fi = [0, 0, 0, 0, 0, 0, 0, 0, 0]

In [81]:
flist = [[], [], [], [], [], [], [], [], []]
for p in project_list:
    data = combined_for_project[p]
    for x in range(0,9):
        flist[x].append(data[x])

In [82]:
for i in range(0, 9):
    final_fi[i] = max(set(flist[i]), key = flist[i].count)

In [83]:
#after manual interference
final_fi[4] = 2

In [84]:
final_fi

[7, 8, 0, 5, 2, 6, 3, 4, 1]

7 gh_num_commits_on_files_touched
8 time_out
0 git_diff_src_churn
5 gh_diff_files_modified
2 git_diff_test_churn
6 git_num_all_built_commits
3 gh_diff_files_added
4 gh_diff_files_deleted
1 gh_num_commit_comments


In [91]:
final_fi_values = [0, 0, 0, 0, 0, 0, 0, 0, 0]
fvalue = [[], [], [], [], [], [], [], [], []]

for p in values_for_projects:
    for x in range(0,9):
        fvalue[x].append(values_for_projects[p][x])

for i in range(0, 9):
    final_fi_values[i] = median(fvalue[i])

In [96]:
for i in range(0, 9):
    print('{:>5} {:>40} {:>30}'.format(final_fi[i], feature_names[final_fi[i]], final_fi_values[final_fi[i]]))

    7          gh_num_commits_on_files_touched            0.29517588602729683
    8                                 time_out            0.20345374087787974
    0                       git_diff_src_churn            0.18473665118874483
    5                   gh_diff_files_modified            0.09840918800266697
    2                      git_diff_test_churn            0.08877735422697038
    6                git_num_all_built_commits           0.043456895082151004
    3                      gh_diff_files_added           0.039224786758027735
    4                    gh_diff_files_deleted            0.01814480786037624
    1                   gh_num_commit_comments           0.005835691072424722
